In [1]:
from data.download_data import *

In [2]:
dd = DownloadData()

In [3]:
dd.download_kaggle()

Starting pariza/bbc-news-summary Download
Completed pariza/bbc-news-summary Download
Starting rmisra/news-category-dataset Download
Completed rmisra/news-category-dataset Download


In [4]:
dd.json_to_df()

In [5]:
dd.download_to_text()

In [6]:
dd.move_rename_files()

Moved and renamed data/raw/BBC News Summary/News Articles/business\001.txt to data/processed/train/text\00009.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\002.txt to data/processed/train/text\00010.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\003.txt to data/processed/train/text\00011.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\004.txt to data/processed/train/text\00012.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\005.txt to data/processed/train/text\00013.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\006.txt to data/processed/train/text\00014.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\007.txt to data/processed/train/text\00015.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\008.txt to data/processed/train/text\00016.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\009.txt to data/proce

In [33]:
train_df

,link,headline,category,short_description,authors,date
0,https://www.huffingtonpost.com/entry/irs-refun...,IRS Refund 2013: Here's What You Need To Know,BUSINESS,Remember: Tax Day 2013 -- the deadline to file...,Harry Bradford,2013-04-06
1,https://www.huffingtonpost.comhttp://www.theat...,Who Are the Long-Term Unemployed?,BUSINESS,It's been over four years since the recovery o...,,2013-08-25
2,https://www.huffingtonpost.com/entry/uber-repa...,Uber Vows To Repay NYC Drivers 'Tens Of Millio...,BUSINESS,The company says it miscalculated its commissi...,Ryan Grenoble,2017-05-24
3,https://www.huffingtonpost.comhttp://www.nytim...,How Colleges Are Failing Low-Income Students,BUSINESS,Most low-income students who have top test sco...,,2013-03-17
4,https://www.huffingtonpost.comhttp://247wallst...,Companies With Record Layoffs In 2012: 24/7 Wa...,BUSINESS,Whatever recovery the jobs market posted in 20...,,2013-01-13
...,...,...,...,...,...,...
4189,https://www.huffingtonpost.com/entry/i-want-to...,I Want to File Bankruptcy,BUSINESS,Huffington Post Reader Question Tamara If you'...,"Steve Rhode, Contributor\nConsumer Debt Expert...",2013-05-25
4190,https://www.huffingtonpost.com/entry/recycling...,Recycling Opens the Door to a Circular Economy,BUSINESS,"Recycling is critically important, but it's on...","Brian Kennell, ContributorPresident and CEO of...",2014-11-15
4191,https://www.huffingtonpost.com/entry/amazon-pr...,Amazon Prime Now Drivers Claim They Were Paid ...,BUSINESS,"Who's paying for the delivery service? ""Down-o...",Dave Jamieson,2015-10-28
4192,https://www.huffingtonpost.com/entry/how-weara...,How Wearable Technology Will Change Social Media,BUSINESS,The ways we use a smart watch (or other wearab...,"Larry Alton, ContributorBlogger, writer, busin...",2015-03-17


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

raw_df = pd.read_json('data/raw/News_Category_Dataset_v3.json', lines=True)
df = raw_df[raw_df['category'] == 'BUSINESS']
df = df.reset_index(drop=True)

train_ratio = 0.7  # Percentage of data for training set
val_ratio = 0.15  # Percentage of data for validation set
test_ratio = 0.15  # Percentage of data for test set

# Splitting into train and remaining data
train_df, remaining_df = train_test_split(df, test_size=(1 - train_ratio), random_state=0)

# Splitting remaining data into validation and test sets
val_df, test_df = train_test_split(remaining_df, test_size=test_ratio/(test_ratio + val_ratio), random_state=0)

# Optional: Reset the index for each DataFrame
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [4]:
dataframes = {'train': train_df, 'val': val_df, 'test': test_df}

In [9]:
import requests
from bs4 import BeautifulSoup
import os

def get_text(max_n = 10):

    processed_dir = 'data/processed/'

    for df_name, df in dataframes.items():

        if not os.path.exists(processed_dir + df_name):
            # Create the directory
            os.makedirs(processed_dir + df_name)
            os.makedirs(processed_dir + df_name + '/text')
            os.makedirs(processed_dir + df_name + '/summaries')

        elif os.listdir(processed_dir + df_name + '/text') or os.listdir(processed_dir + df_name + '/summaries'):
            continue

        for idx, row in df.iterrows():

            if idx == max_n:
                break

            filename = f"{idx:05d}.txt"

            url = row.link
            summary = row.short_description

            # Check if the substring is present in the string
            substring = "http"

            # Check if either substring is present in the string
            if substring in url:
                # Find the indices of the last occurrences of the substrings
                index = url.rfind(substring)
                url = url[index:]

            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')

            # Extract body text from article
            bodytext = soup.find_all('p')
            bodytext = [i.text for i in bodytext]
            text = ' '.join(bodytext)

            with open(f'{processed_dir}{df_name}/text/' + filename, 'w', encoding='utf-8') as file:
                file.write(text)

            with open(f'{processed_dir}{df_name}/summaries/' + filename, 'w', encoding='utf-8') as file:
                file.write(summary)

In [47]:
get_text()

In [29]:
train_df

,link,headline,category,short_description,authors,date
0,https://www.huffingtonpost.com/entry/irs-refun...,IRS Refund 2013: Here's What You Need To Know,BUSINESS,Remember: Tax Day 2013 -- the deadline to file...,Harry Bradford,2013-04-06
1,https://www.huffingtonpost.comhttp://www.theat...,Who Are the Long-Term Unemployed?,BUSINESS,It's been over four years since the recovery o...,,2013-08-25
2,https://www.huffingtonpost.com/entry/uber-repa...,Uber Vows To Repay NYC Drivers 'Tens Of Millio...,BUSINESS,The company says it miscalculated its commissi...,Ryan Grenoble,2017-05-24
3,https://www.huffingtonpost.comhttp://www.nytim...,How Colleges Are Failing Low-Income Students,BUSINESS,Most low-income students who have top test sco...,,2013-03-17
4,https://www.huffingtonpost.comhttp://247wallst...,Companies With Record Layoffs In 2012: 24/7 Wa...,BUSINESS,Whatever recovery the jobs market posted in 20...,,2013-01-13
...,...,...,...,...,...,...
4189,https://www.huffingtonpost.com/entry/i-want-to...,I Want to File Bankruptcy,BUSINESS,Huffington Post Reader Question Tamara If you'...,"Steve Rhode, Contributor\nConsumer Debt Expert...",2013-05-25
4190,https://www.huffingtonpost.com/entry/recycling...,Recycling Opens the Door to a Circular Economy,BUSINESS,"Recycling is critically important, but it's on...","Brian Kennell, ContributorPresident and CEO of...",2014-11-15
4191,https://www.huffingtonpost.com/entry/amazon-pr...,Amazon Prime Now Drivers Claim They Were Paid ...,BUSINESS,"Who's paying for the delivery service? ""Down-o...",Dave Jamieson,2015-10-28
4192,https://www.huffingtonpost.com/entry/how-weara...,How Wearable Technology Will Change Social Media,BUSINESS,The ways we use a smart watch (or other wearab...,"Larry Alton, ContributorBlogger, writer, busin...",2015-03-17


In [ ]:
import os
import shutil

max_n = 10

source_directory = "data/raw/BBC News Summary/BBC News Summary/News Articles/business"
target_directory = "data/processed/train"

# Get the list of files in the source directory
file_list = os.listdir(source_directory)

# Filter the list to include only text files
txt_files = [file for file in file_list if file.endswith(".txt")]

# Sort the list of text files in ascending order
txt_files.sort()

# Move and rename the files
for index, file_name in enumerate(txt_files, start=largest_name_int):
    if index == max_n:
        break
    source_path = os.path.join(source_directory, file_name)
    target_name = f"{index:05}.txt"
    target_path = os.path.join(target_directory, target_name)
    shutil.move(source_path, target_path)
    print(f"Moved and renamed {file_name} to {target_name}")


In [21]:
def get_largest_name_int(example_dir = "data/processed/train/text/"):
    # Get the list of files in the directory
    file_list = os.listdir(example_dir)

    # Filter the list to include only text files
    txt_files = [file for file in file_list if file.endswith(".txt")]

    # Sort the list of text files in ascending order
    txt_files.sort()

    # Get the largest name
    largest_name = txt_files[-1]

    # Convert the largest name to an integer
    largest_name_int = int(largest_name.split(".")[0])

    return largest_name_int

def move_rename_files(max_n = 10):
    
    largest_name_int = get_largest_name_int()

    dir_tuple = [("data/raw/BBC News Summary/News Articles/business", "text"), 
                 ("data/raw/BBC News Summary/Summaries/business", "summaries")]

    # Move and rename the files
    for df_name in dataframes.keys():

        for i in range(len(dir_tuple)):

            source_directory, target_ext = dir_tuple[i]

            target_directory = f"data/processed/{df_name}/{target_ext}"
            text_files = [file for file in os.listdir(source_directory) if file.endswith(".txt")]

            for index, file_name in enumerate(text_files, start=largest_name_int):

                if index == max_n + largest_name_int:
                    break
                
                source_path = os.path.join(source_directory, file_name)
                target_name = f"{index:05}.txt"
                target_path = os.path.join(target_directory, target_name)
                shutil.move(source_path, target_path)
                print(f"Moved and renamed {source_path} to {target_path}")

In [24]:
move_rename_files()

Moved and renamed data/raw/BBC News Summary/News Articles/business\103.txt to data/processed/train/text\00036.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\104.txt to data/processed/train/text\00037.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\105.txt to data/processed/train/text\00038.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\106.txt to data/processed/train/text\00039.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\107.txt to data/processed/train/text\00040.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\108.txt to data/processed/train/text\00041.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\109.txt to data/processed/train/text\00042.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\110.txt to data/processed/train/text\00043.txt
Moved and renamed data/raw/BBC News Summary/News Articles/business\111.txt to data/proce

In [3]:
import os